# Ch_06_학습 관련 기술들

6장에서는 신경망의 학습 효율을 높이기 위한 방법들을 설명하고자 한다.

설명할 내용들은 다음과 같다.

최적의 가중치 매개변수를 찾기 위한 최적화 방법.

초기 가중치 매개변수 설정에 관한 방법.

오버피팅 조정하는 방법.

배치 정규화.

6.1 매개변수 갱신

신경망 학습의 원리는 손실함수라는 지표를 정의한 다음에 손실함수 값을 가장 적게 하는 최적의 가중치 매개변수를 구하는 것이었다.

이처럼 어떤 목적함수의 최솟값을 찾는 문제를 수학에서는 최적화(Optimization)라고 한다.

하지만, 실제 feasible set의 영역이 매우 넓고, 변수 또한 많기 때문에 쉽게 계산 되지 않는다.

앞 단원에서 매개변수에 대한 손실함수의 기울기인 Gradient를 이용한 SGD(Stochastic Gradient Descent,확률적 경사 하강법)을 공부했다.

SGD는 처음 최적화를 풀 때 가장 자연스러운 방법이지만, 한계가 존재한다. 장단점을 살펴 본 후 보완할 수 있는 방법을 살펴보자.

6.1.1 모험가 이야기

어떤 모험가가 앞을 볼 수 없는 상태에서 현재의 위치로부터 가장 낮은 곳으로 가기 위해선 어떻게 해야할까?

지금 있는 곳에서 가장 땅의 기울기가 크게 기울어지는 곳으로 가면 될 것 같다.

6.1.2 SGD

SGD의 기본 원리를 복기 해보자.

현재의 가중치 행렬을 $\mathbf{W}$라 하면, 학습률 $\eta$에 대해 새로운 가중치를 다음과 같이 업데이트 한다.

[식 6.1]$~~~~\mathbf{W} = \mathbf{W} - \eta \frac{ \partial{L}}{\partial{W}} $

간단한 SGD를 구현해보자.

In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads): #성분 별이 아닌 행렬 통째로 업데이트 함.
        for key in params.keys():
            params[key] -= self.lr * grads[key]

코드를 살펴보면, SGD라는 클래스 속의 update라는 Method는 params, grads라는 것을 인수로 받는데, params['W1']과 같이 행렬형태로 저장이 되기 때문에 업데이트가 행렬 통째로 됨을 짐작 할 수 있다.

이 SGD 클래스를 이용하여 앞에서의 신경망에서 매개변수 갱신 부분에 사용하면 된다.

6.1.3 SGD의 단점

개념 자체가 자연스럽고, 계산도 그리 복잡하지 않음에도 불구하고 SGD가 널리 쓰이지 않는 이유가 있다.

다음과 같은 식을 생각해보자.

[식 6.2]
![](data/images/e%206.2.png)
위의 식을 그래프로 그리면 다음과 같다.
![](data/images/fig%206-1.png)
이 때 [식 6.2]의 각 점에서의 Gradient를 살펴보면,
![](data/images/fig%206-2.png)
즉, y축 방향은 크게 감소하는데 x축 방향은 크게 바뀌지 않는다. 그리고 Global minimum 값은 (0,0)에서 나타나는게 자명하지만, Gradient는 모두 (0,0)을 가리키진 않는다. 이런 함수에 대해 SGD를 한번 적용해보자.

초기값을 (-7,2)로 했을 때 SGD 적용 결과는
![](data/images/fig%206-3.png)
즉, global minimum인 (0,0)으로 가긴 하지만, 지그재그 형태로 매우 많은 반복횟수를 거쳐 감을 알 수 있다.

이런 지그재그 현상이 일어나는 근본적인 원인은 각 점에서의 Gradient의 방향이 Global Minimum의 방향이 아니기 때문에 일어난다.

이런 현상을 개선하기 위해 모멘텀(Momentum), AdaGard, Adam 3가지 방법을 소개한다.

6.1.4 모멘텀

모멘텀은 물리에서 '운동량'에 해당하는 개념으로...